# Recommender System for movies

In this notebook we will create a Recommender System for movies using a Restricted Boltzman Machine. 
The Restricted Boltzman Machine is a Generative Model, with undirectional edges, composed of visible and hidden nodes.
We can see the structure of the network in this picture:

<img src="restrictedBoltzman.png">

In particular, in this model we will use the following datasets:

https://grouplens.org/datasets/movielens/1m/

https://grouplens.org/datasets/movielens/100k/

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
 * 100,000 ratings (1-5) from 943 users on 1682 movies. 
 * Each user has rated at least 20 movies. 
 * Simple demographic info for the users (age, gender, occupation, zip)


## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Dataset

In [14]:
# Movies Data
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['MovieID', 'Title', 'Genre' ])
movies.head()

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [16]:
# Users Data
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['UserID', 'Gender', 'Age', 'Job', 'ZipCode' ])
users.head()

,UserID,Gender,Age,Job,ZipCode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [18]:
# Ratings Data
ratings = pd.read_csv('ml-1m/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1', names = ['UserID', 'MovieID', 'Rating', 'Timestamp' ])
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
